# CommonRoad Scenario Designer Tutorial
This tutorial shows how to use the map conversion APIs of the CommonRoad Scenario Designer.

## 1. Preparation
Before you proceed any further, make sure you have successfully installed the CommonRoad Scenario Designer
(cf. [README](../README.md)).

In [6]:
# General Imports
import os
from lxml import etree

from commonroad.scenario.scenario import Tag
from commonroad.common.file_writer import CommonRoadFileWriter, OverwriteExistingFile
from commonroad.planning.planning_problem import PlanningProblemSet
from commonroad.common.file_reader import CommonRoadFileReader

## 2. OSM to CommonRoad
### Download Map

In [7]:
from crdesigner.conversion.osm2cr.converter_modules.osm_operations.downloader import download_around_map
from crdesigner.conversion.osm2cr import config

download_around_map(config.BENCHMARK_ID + '_downloaded.osm', 48.140289, 11.566272)

downloading map


KeyboardInterrupt: 

### General API

In [ ]:
from crdesigner.io.api import osm_to_commonroad

# load OpenDRIVE file, parse it, and convert it to a CommonRoad scenario
scenario = osm_to_commonroad(config.SAVE_PATH + config.BENCHMARK_ID + ".osm")

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_OSM-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

### OSM conversion APIs

In [ ]:
import crdesigner.conversion.osm2cr.converter_modules.converter as converter
import crdesigner.conversion.osm2cr.converter_modules.cr_operations.export as ex


# open the map and convert it to a scenario
scenario = converter.GraphScenario(config.SAVE_PATH + config.BENCHMARK_ID + "_downloaded.osm")

# draw and show the scenario
scenario.plot()

# save the scenario as commonroad file
scenario.save_as_cr(config.SAVE_PATH + config.BENCHMARK_ID + ".xml")
# save the scenario as a binary
scenario.save_to_file(config.SAVE_PATH + config.BENCHMARK_ID + ".pickle")

# view the generated
ex.view_xml(config.SAVE_PATH + config.BENCHMARK_ID + ".xml")

## 3. OpenDRIVE to CommonRoad

In [ ]:
input_path = ""  # replace empty string

### General API

In [ ]:
from crdesigner.io.api import opendrive_to_commonroad

# load OpenDRIVE file, parse it, and convert it to a CommonRoad scenario
scenario = opendrive_to_commonroad(input_path)

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_OpenDRIVETest-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

### OpenDRIVE conversion APIs

In [ ]:
from crdesigner.conversion.opendrive.opendriveparser.parser import parse_opendrive
from crdesigner.conversion.opendrive.opendriveconversion.network import Network

# OpenDRIVE parser to load file
with open("{}".format(input_path), "r") as file_in:
    opendrive = parse_opendrive(etree.parse(file_in).getroot())

# create OpenDRIVE intermediate network object
road_network = Network()

# convert OpenDRIVE file
road_network.load_opendrive(opendrive)

# export to CommonRoad scenario
scenario = road_network.export_commonroad_scenario()

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_OpenDRIVETest-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

## 4. Lanelet/Lanelet2 to CommonRoad and CommonRoad to Lanelet

In [ ]:
input_path = ""  # replace empty string
proj = ""  # replace empty string
left_driving = False  # replace with favoured value
adjacencies = False  # replace with favoured value

### General API

In [ ]:
from crdesigner.io.api import lanelet_to_commonroad

scenario = lanelet_to_commonroad(input_path, proj, left_driving, adjacencies)

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_Lanelet-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

### Lanelet conversion APIs

In [ ]:
from crdesigner.conversion.lanelet_lanelet2.lanelet2cr import Lanelet2CRConverter
from crdesigner.conversion.lanelet_lanelet2.lanelet2_parser import Lanelet2Parser

# read and parse lanelet/lanelet2 file
parser = Lanelet2Parser(etree.parse(input_path).getroot())
lanelet2_content = parser.parse()

# convert lanelet/lanelet2 map to CommonRoad
lanelet2_converter = Lanelet2CRConverter(proj_string=proj)
scenario = lanelet2_converter(lanelet2_content, detect_adjacencies=adjacencies, left_driving_system=left_driving)

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_Lanelet-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

## 5. CommonRoad to Lanelet

In [ ]:
input_path = ""  # replace empty string
output_name = ""  # replace empty string
proj = ""  # replace empty string

### General API

In [ ]:
from crdesigner.io.api import commonroad_to_lanelet

# load CommonRoad file and convert it to lanelet format
commonroad_to_lanelet(input_path, output_name, proj)

### Lanelet conversion APIs

In [ ]:
from crdesigner.conversion.lanelet_lanelet2.cr2lanelet import CR2LaneletConverter

try:
    commonroad_reader = CommonRoadFileReader(input_path)
    scenario, _ = commonroad_reader.open()
except etree.XMLSyntaxError as xml_error:
    print(f"SyntaxError: {xml_error}")
    print("There was an error during the loading of the selected CommonRoad file.\n")
    scenario = None

if scenario:
    l2osm = CR2LaneletConverter(proj)
    osm = l2osm(scenario)
    with open(f"{output_name}", "wb") as file_out:
        file_out.write(etree.tostring(osm, xml_declaration=True, encoding="UTF-8", pretty_print=True))

## 6. SUMO to CommonRoad

In [ ]:
input_file = ""  # replace empty string


### General API


In [ ]:
from crdesigner.io.api import sumo_to_commonroad

# load SUMO net file, parse it, and convert it to a CommonRoad map
scenario = sumo_to_commonroad(input_file)

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_SUMO-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

### SUMO conversion APIs


In [ ]:
from crdesigner.io.gui.toolboxes.gui_sumo_simulation import SUMO_AVAILABLE
if SUMO_AVAILABLE:
    from crdesigner.conversion.sumo_map.sumo2cr import convert_net_to_cr

# load SUMO net file, parse it, and convert it to a CommonRoad map
scenario = convert_net_to_cr(input_file)

# store converted file as CommonRoad scenario
writer = CommonRoadFileWriter(
    scenario=scenario,
    planning_problem_set=PlanningProblemSet(),
    author="Sebastian Maierhofer",
    affiliation="Technical University of Munich",
    source="CommonRoad Scenario Designer",
    tags={Tag.URBAN},
)
writer.write_to_file(os.path.dirname(os.path.realpath(__file__)) + "/" + "ZAM_SUMO-1_1-T1.xml",
                     OverwriteExistingFile.ALWAYS)

## 7. CommonRoad to SUMO

In [ ]:
output_folder = ""  # replace empty string
scenario_name = ""  # replace empty string
input_file = os.path.join(output_folder, scenario_name + '.xml')
scenario, planning_problem = CommonRoadFileReader(input_file).open()

### General API

In [ ]:
from crdesigner.io.api import commonroad_to_sumo

commonroad_to_sumo(input_file, output_folder)

### SUMO conversion APIs


In [ ]:
import uuid
from crdesigner.io.gui.toolboxes.gui_sumo_simulation import SUMO_AVAILABLE
if SUMO_AVAILABLE:
    from crdesigner.conversion.sumo_map.config import SumoConfig
    from crdesigner.conversion.sumo_map.cr2sumo import CR2SumoMapConverter

try:
    commonroad_reader = CommonRoadFileReader(input_file)
    scenario, _ = commonroad_reader.open()
except etree.XMLSyntaxError as xml_error:
    print(f"SyntaxError: {xml_error}")
    print(
        "There was an error during the loading of the selected CommonRoad file.\n")

if SUMO_AVAILABLE:
    config = SumoConfig.from_scenario_name(str(uuid.uuid4()))
    config.scenario_name = scenario_name
    converter = CR2SumoMapConverter(scenario.lanelet_network, config)
    converter.convert_to_net_file(output_folder)

### SUMO conversion APIs with Traffic Simulation and Video Creation

In [ ]:
from commonroad.visualization.draw_dispatch_cr import draw_object
import matplotlib.pyplot as plt
import numpy as np
from crdesigner.io.gui.toolboxes.gui_sumo_simulation import SUMO_AVAILABLE
if SUMO_AVAILABLE:
    from crdesigner.conversion.sumo_map.config import SumoConfig
    from crdesigner.conversion.sumo_map.cr2sumo.converter import CR2SumoMapConverter
    from sumocr.interface.sumo_simulation import SumoSimulation
    from sumocr.visualization.video import create_video

# translate scenario to center
centroid = np.mean(np.concatenate(
    [la.center_vertices for la in scenario.lanelet_network.lanelets]),
    axis=0)
scenario.translate_rotate(-centroid, 0)
planning_problem.translate_rotate(-centroid, 0)

config = SumoConfig.from_scenario_name(scenario_name)

# convert CR to sumo net
wrapper = CR2SumoMapConverter(scenario.lanelet_network, config)
wrapper.convert_to_net_file(output_folder)
tls_lanelet_id = 43513
traffic_light_system_generated = wrapper.auto_generate_traffic_light_system(tls_lanelet_id)

print(f"Generated Traffic Light System at {tls_lanelet_id}, {traffic_light_system_generated}")

# draw scenario after traffic light generation
plt.figure(figsize=(25, 25))
draw_object(wrapper.lanelet_network)
plt.axis('equal')
plt.autoscale()
plt.show()

# write generated traffic lights back to commonroad file
scenario.lanelet_network = wrapper.lanelet_network

# run Simulation
simulation = SumoSimulation()
simulation.initialize(config, wrapper)

for t in range(config.simulation_steps):
    simulation.simulate_step()

simulation.stop()

# save resulting scenario
simulated_scenario = simulation.commonroad_scenarios_all_time_steps()
CommonRoadFileWriter(simulated_scenario,
                     planning_problem,
                     author=scenario.author,
                     affiliation=scenario.affiliation,
                     source=scenario.source,
                     tags=scenario.tags,
                     location=scenario.location).write_scenario_to_file(
    os.path.join(output_folder,
                 config.scenario_name + ".simulated.xml"),
    overwrite_existing_file=OverwriteExistingFile.ALWAYS)

print("creating video (this may take some time)")
create_video(simulation, 1, config.simulation_steps, output_folder)